# 10. Hafta Lab-4: LSTM Sınıflandırma
## BGM 565: Siber Güvenlik için Makine Öğrenme Yöntemleri
### İstanbul Şehir Üni. - Bilgi Güvenliği Müh.
#### Dr. Ferhat Özgür Çatak
Bu lab çalışması kapsamında lab-2'de kullanılan dökümanların **tf-idf** matrisi yerine lstm algoritması ile sınıflandırma modeli oluşturulacaktır.

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import BaggingClassifier
import numpy as np
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

# np.set_printoptions(threshold=np.nan)
np.set_printoptions(linewidth=160)

Using TensorFlow backend.


### Örnek veri kümesi 20 newsgroup kullanılacaktır.

In [2]:
docs = fetch_20newsgroups(subset='train', shuffle=True)


EOFError: Compressed file ended before the end-of-stream marker was reached

**one_hot** encoding yöntemi kullanılarak her bir kelime *integer encoding* oluşturulacaktır.

In [ ]:
vocab_size = 100000
encoded_docs = [one_hot(d, vocab_size) for d in docs.data]

In [ ]:
max_length = 4500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

### Deep Neural Network Sınıflandırma Modeli

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.layers.embeddings import Embedding
from keras.layers import Flatten

X_train, X_test, y_train, y_test = train_test_split(padded_docs, docs.target, test_size=0.7, random_state=42)

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
dummy_Y = np_utils.to_categorical(encoded_Y)
print(encoded_Y.shape)

In [ ]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(LSTM(8, return_sequences=True))
model.add(Flatten())
#model.add(LSTM(8))
model.add(Dense(len(docs.target_names), activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
model.summary()

In [ ]:
history = model.fit(X_train, dummy_Y, epochs=20, batch_size=50, verbose=1,
                    validation_split=0.3)